torch_geometric.data.Dataset

torch_geometric.data.InMemoryDataset第二个是第一个的子类，如果希望全部数据都在内存里则需要使用第二个类

要创建一个名为"InMemoryDataset"的数据集类，你需要实现以下四个基本方法：

raw_file_names(raw_dir)：该方法需要返回原始文件在raw_dir目录中的列表。如果原始文件已经存在于该目录中，则下载过程可以跳过。

processed_file_names(processed_dir)：这个方法需要返回处理后文件在processed_dir目录中的列表。这将用于跳过处理过程。

download(raw_dir)：这个方法用于将原始文件下载到raw_dir目录中。

process(raw_data, processed_dir)：这个方法用于处理原始数据并将其保存到processed_dir目录中。在这个方法中，你需要读取原始数据并创建一个数据对象列表，然后将其保存到文件夹中。为了提高存储效率，你可以使用collate方法将数据对象列表合并为一个大的数据对象，然后从该对象中生成一个slices字典，用于重建单个样例。最后，你需要加载两个对象self.data和self.slices。

In [ ]:
import torch
from torch_geometric.data import InMemoryDataset, download_url


class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super(MyOwnDataset, self).__init__(root, transform, pre_transform, pre_filter)
        # 初始化时将数据读入内存
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def download(self):
        # Download to 'self.raw_file'
        download_url(url, self.raw_dir) # 这里的url需要自己指定

    def process(self):
        # Read data into huge 'Data' list
        data_list = [...]
        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]
        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        # 将处理后的数据存储
        torch.save((data, slices), self.processed_paths[0])


更大数据集

In [ ]:
import os.path as osp

import torch
from torch_geometric.data import Dataset, download_url


class MyOwnDataset(Dataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data_1.pt', 'data_2.pt', ...]

    def download(self):
        # Download to `self.raw_dir`.
        path = download_url(url, self.raw_dir)
        ...
    ## 上面是一样的，重点是下面
    def process(self):
    	# 因为数据比较多，无法一次读入内存，所以以图为单位分开读取、处理、再存储
        idx = 0
        for raw_path in self.raw_paths:
            # Read data from `raw_path`.
            data = Data(...)

            if self.pre_filter is not None and not self.pre_filter(data):
                continue

            if self.pre_transform is not None:
                data = self.pre_transform(data)

            torch.save(data, osp.join(self.processed_dir, f'data_{idx}.pt'))
            idx += 1

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
        return data
